In [ ]:
# Jupyter Notebook Code
from code_gen_result_analysis import CodeAnalysis, cal_err_bar, bootstrap_resampling  # Assuming code_analysis.py is the name of the file
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Function to style the DataFrame
def highlight_cols(df):
     # We will create a custom style for the headers and vertical lines
     styles = [
               dict(selector="th.col_heading",
                    props=[("text-align", "center"),
                              ("border-right", "2px solid #6c6c6c"),
                              ("border-left", "2px solid #6c6c6c")]),
               dict(selector="th.col_heading.level0",
                    props=[("border-top", "2px solid #6c6c6c")]),
               dict(selector="th",
                    props=[("font-size", "12px")]),
               dict(selector="td",
                    props=[("text-align", "center")]),
               # Customize the boundary for your specific DataFrame structure
               # Adjust "4" and "10" according to your DataFrame's column indices
               dict(selector=f"th:nth-child(5), td:nth-child(5)",
                    props=[("border-left", "2px solid #6c6c6c")]),
               dict(selector=f"th:nth-child(11), td:nth-child(11)",
                    props=[("border-left", "2px solid #6c6c6c")])
          ]
     return df.style.set_table_styles(styles).set_properties(**{'width': '120px', 'text-align': 'center'}).hide_index()

label_mapping = {
     'If Condition': 'If Condition', #If Condition
     'Elif Condition': 'If Condition',
     'If Body': 'If Body',
     'Elif Body': 'If Body',
     'Else Reasoning': 'If Body',
     'Loop Body': 'Loop Body',
     'Define Stop Criteria': 'Loop Body',
     'List Comprehension': 'List Comprehension',
     'Lambda Expressions': 'List Comprehension',
     'Generator Expressions': 'List Comprehension',
}
def get_summary_df(directory_path, label_mapping, weighted=False, use_max_range=True,gen_code_pass_col = 'post_process_pass_ratio'):
     analysis = CodeAnalysis(weighted=weighted, use_max_range=use_max_range, gen_code_pass_col=gen_code_pass_col)  # Pass additional parameters if different from defaults
     # Analyze results and structure the summary DataFrame
     summary_df = analysis.analyze_results_in_folder(directory_path, label_mapping=label_mapping)
     # Sort the DataFrame by 'Model Name'
     summary_df_sorted = summary_df.sort_values(by='Model Name')
     # Creating MultiIndex for columns to include subheaders
     reasons_columns = [('Reason Categories Pass Ratio', col.replace('_', ' ')) for col in analysis.labels_reasons]
     horizons_columns = [('Horizon Categories Pass Ratio', col.replace('_', ' ')) for col in analysis.labels_horizons]
     columns = [('General', 'Model Name'), ('General', 'Generation Mode'), ('General', 'Code Task'), ('General', 'All Pass Ratio')] + reasons_columns + horizons_columns
     summary_df_sorted.columns = pd.MultiIndex.from_tuples(columns)

     # Apply the styling function to your summary DataFrame
     styled_df = highlight_cols(summary_df_sorted)
     return summary_df_sorted, styled_df

# Function to split the ratio column into 'Passed Count' and 'Total Count'
def split_ratio_for_general(df, debug_print=False):
     if debug_print:
          print(df[('General', 'All Pass Ratio')])  # Print before split
     df[('General', 'Passed Count')] = df[('General', 'All Pass Ratio')].str.extract('\((\d+)/').squeeze().astype(int)
     df[('General', 'Total Count')] = df[('General', 'All Pass Ratio')].str.extract('/(\d+)\)').squeeze().astype(int)
     if debug_print:
          print(df[('General', 'Passed Count')])
          print(df[('General', 'Total Count')])
     return df

# Function to analyze the DataFrame by 'Model Name' and 'Generation Mode'
def analyze_by_model_and_mode_for_general(df, report_err_bar=True):
     df = split_ratio_for_general(df)
     grouped_df = df.groupby([('General', 'Model Name'), ('General', 'Generation Mode')]).sum()

     total_counts = grouped_df[('General', 'Total Count')]
     pass_counts = grouped_df[('General', 'Passed Count')]

     percentages = pass_counts / total_counts * 100

     if report_err_bar:
          percentages, err_bar = cal_err_bar(pass_counts, total_counts)
          err_bar = pd.Series(err_bar, index=grouped_df.index)
          grouped_df[('General', 'All Pass Ratio')] = grouped_df.apply(lambda row: f"{percentages.loc[row.name]*100:.1f} ± {err_bar.loc[row.name]*100:.1f}", axis=1)
     else:
          grouped_df[('General', 'All Pass Ratio')] = percentages.map("{:.3f}%".format)

     return grouped_df

# Function to split the ratio column into 'Passed Count' and 'Total Count'
# for Reason Categories Pass Ratio and Horizon Categories Pass Ratio
def split_ratio(df):
     reason_categories = ['If Condition','If Body', 'Stream Operations', 'Loop Body', 'Super Call']
     horizon_categories = ['Short-Range', 'Medium-Range', 'Long-Range', 'Variable', 'Global Variable', 'Function', 'Class', 'Library', 'Interface']

     for category in reason_categories:
          df[('Reason Categories Pass Ratio', category + ' Passed Count')] = df[('Reason Categories Pass Ratio', category)].str.extract('\((\d+)/').squeeze().fillna(0).astype(int)
          df[('Reason Categories Pass Ratio', category + ' Total Count')] = df[('Reason Categories Pass Ratio', category)].str.extract('/(\d+)\)').squeeze().fillna(0).astype(int)

     for category in horizon_categories:
          df[('Horizon Categories Pass Ratio', category + ' Passed Count')] = df[('Horizon Categories Pass Ratio', category)].str.extract('\((\d+)/').squeeze().fillna(0).astype(int)
          df[('Horizon Categories Pass Ratio', category + ' Total Count')] = df[('Horizon Categories Pass Ratio', category)].str.extract('/(\d+)\)').squeeze().fillna(0).astype(int)

     return df

def analyze_by_model_and_mode(df, report_err_bar=True):
     df = split_ratio(df)
     grouped_df = df.groupby([('General', 'Model Name'), ('General', 'Generation Mode')]).sum()

     horizon_categories = ['Short-Range', 'Medium-Range', 'Long-Range', 'Variable', 'Global Variable', 'Function', 'Class', 'Library', 'Interface']
     reason_categories = ['If Condition','If Body', 'Stream Operations', 'Loop Body', 'Super Call']

     if report_err_bar:
          for category in horizon_categories:
               total_counts = grouped_df[('Horizon Categories Pass Ratio', category + ' Total Count')]
               pass_counts = grouped_df[('Horizon Categories Pass Ratio', category + ' Passed Count')]
               percentages = pass_counts / total_counts * 100

               if report_err_bar:
                    percentages, err_bar = cal_err_bar(pass_counts, total_counts)
                    err_bar = pd.Series(err_bar, index=grouped_df.index)
                    grouped_df[('Horizon Categories Pass Ratio', category)] = grouped_df.apply(lambda row: f"{percentages.loc[row.name]*100:.1f} ± {err_bar.loc[row.name]*100:.1f}", axis=1)
               else:
                    grouped_df[('Horizon Categories Pass Ratio', category)] = percentages.map("{:.3f}%".format)

               grouped_df.drop([('Horizon Categories Pass Ratio', category + ' Passed Count'), ('Horizon Categories Pass Ratio', category + ' Total Count')], axis=1, inplace=True)

          for category in reason_categories:
               total_counts = grouped_df[('Reason Categories Pass Ratio', category + ' Total Count')]
               pass_counts = grouped_df[('Reason Categories Pass Ratio', category + ' Passed Count')]
               percentages = pass_counts / total_counts * 100

               if report_err_bar:
                    percentages, err_bar = cal_err_bar(pass_counts, total_counts)
                    err_bar = pd.Series(err_bar, index=grouped_df.index)
                    grouped_df[('Reason Categories Pass Ratio', category)] = grouped_df.apply(lambda row: f"{percentages.loc[row.name]*100:.1f} ± {err_bar.loc[row.name]*100:.1f}", axis=1)
               else:
                    grouped_df[('Reason Categories Pass Ratio', category)] = percentages.map("{:.3f}%".format)

               grouped_df.drop([('Reason Categories Pass Ratio', category + ' Passed Count'), ('Reason Categories Pass Ratio', category + ' Total Count')], axis=1, inplace=True)
     else:
          for category in horizon_categories:
               pass_ratio = grouped_df[('Horizon Categories Pass Ratio', category + ' Passed Count')] / grouped_df[('Horizon Categories Pass Ratio', category + ' Total Count')] * 100
               grouped_df[('Horizon Categories Pass Ratio', category)] = pass_ratio.map("{:.3f}%".format) + '(' + grouped_df[('Horizon Categories Pass Ratio', category + ' Passed Count')].astype(str) + '/' + grouped_df[('Horizon Categories Pass Ratio', category + ' Total Count')].astype(str) + ')'
               grouped_df.drop([('Horizon Categories Pass Ratio', category + ' Passed Count'), ('Horizon Categories Pass Ratio', category + ' Total Count')], axis=1, inplace=True)

          for category in reason_categories:
               pass_ratio = grouped_df[('Reason Categories Pass Ratio', category + ' Passed Count')] / grouped_df[('Reason Categories Pass Ratio', category + ' Total Count')] * 100
               grouped_df[('Reason Categories Pass Ratio', category)] = pass_ratio.map("{:.3f}%".format) + '(' + grouped_df[('Reason Categories Pass Ratio', category + ' Passed Count')].astype(str) + '/' + grouped_df[('Reason Categories Pass Ratio', category + ' Total Count')].astype(str) + ')'
               grouped_df.drop([('Reason Categories Pass Ratio', category + ' Passed Count'), ('Reason Categories Pass Ratio', category + ' Total Count')], axis=1, inplace=True)

     return grouped_df

def clean_df(df, drop_general=False, drop_range=True, drop_nan=True, drop_zero=True, col_to_drop=None):
     if drop_general:
          df = df.drop(columns=[col for col in df.columns if 'Passed Count' in col or 'Total Count' in col], errors='ignore')

     if drop_range:
          df = df.drop(columns=[('Horizon Categories Pass Ratio', 'Short-Range'), ('Horizon Categories Pass Ratio', 'Medium-Range'), ('Horizon Categories Pass Ratio', 'Long-Range')], errors='ignore')

     if drop_nan:
          df = df.loc[:, ~(df == 'nan%(0/0)').all()]

     if drop_zero:
          df = df.loc[:, ~(df == '0.0 ± 0.0').all()]
     
     if col_to_drop is not None:
          df = df.drop(columns=[('Horizon Categories Pass Ratio', col_to_drop)], errors='ignore')

     return df

def analyze_and_save(directory_path, label_mapping, save_file_name, gen_code_pass_col='post_process_pass_ratio', col_to_drop=None):
     summary_df_sorted, styled_df = get_summary_df(directory_path, label_mapping, gen_code_pass_col = gen_code_pass_col)
     general_analyzed_df = analyze_by_model_and_mode_for_general(summary_df_sorted)
     display(general_analyzed_df)
     analyzed_df = analyze_by_model_and_mode(summary_df_sorted)
     cleaned_df = clean_df(analyzed_df, col_to_drop=col_to_drop)
     display(cleaned_df)
     cleaned_df.to_csv(save_file_name)

# Set display options for pandas DataFrame
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


In [ ]:
gen_code_pass_col = 'post_process_pass_ratio'  # gen_code_pass_col= 'gen_code_pass_ratio' for results without post-processing
root_path = "../Analysis_Results/storage_server/"  # Update the root_path as needed

def analyze_and_display_results(language, analysis_type, folder_path, save_file):
    print(f"Analyzing {language} {analysis_type}")
    col_to_drop = "Library" if language == "Java" else None
    analyze_and_save(folder_path, label_mapping, save_file, gen_code_pass_col=gen_code_pass_col,  col_to_drop=col_to_drop)
    print(f"Analysis and saving completed for {language} {analysis_type}\n\n")

# Example usage
analyze_and_display_results("Python", "Completion", f"{root_path}Python_all_res/Completion/4th_post_process_reason_update/Update_labels", "Python_Completion_grouped.csv")
analyze_and_display_results("Python", "Infilling", f"{root_path}Python_all_res/Infilling/4th_post_process_reason_update/Update_labels", "Python_Infilling_grouped.csv")
analyze_and_display_results("Java", "Completion", f"{root_path}Java_all_res/Completion/4th_post_process_reason_update/Update_labels", "Java_Completion_grouped.csv")
analyze_and_display_results("Java", "Infilling", f"{root_path}Java_all_res/Infilling/4th_post_process_reason_update/Update_labels", "Java_Infilling_grouped.csv")